In [ ]:
import torch, torchaudio
import IPython.display as ipd
from knn_vc.wav_convert import convert_flac_to_wav

torch.cuda.is_available()

# KNN-VC Example usage

In [ ]:

download_dir = "~/media/Data/CambAI/TakeHome/CambAI_Takehome/knn-vc-downloads"
torch.hub.set_dir(download_dir)
knn_vc = torch.hub.load('bshall/knn-vc', 'knn_vc', prematched=True, trust_repo=True, pretrained=True, device="cuda:0")
# torch.save(knn_vc.state_dict(), "./knn_vc_state_dict")





In [ ]:

src_wav_path = "./datasets/debug_set/srcs/4507-16021-0024.flac" # 16kHz file to be voice-converted
ref_wav_paths = ["./datasets/debug_set/targets/61-70968-0000.flac", "./datasets/debug_set/targets/61-70968-0001.flac", "./datasets/debug_set/targets/61-70968-0002.flac", 
                 "./datasets/debug_set/targets/61-70968-0003.flac"] # list of 16kHz files that serve as the target, the src_wav_path will be converted to this speaker


query_seq = knn_vc.get_features(src_wav_path) # Returns features of `path` waveform as a tensor of shape (seq_len, dim) --> data preprocessing
matching_set = knn_vc.get_matching_set(ref_wav_paths) # Get matching features to be used by wavlm --> Data preprocessing

out_wav = knn_vc.match(query_seq, matching_set, topk=4) # Using matcher to perform VC, basically VC.forward


In [ ]:
ipd.Audio(out_wav.numpy(), rate=16000)
torchaudio.save('knnvc1_out.wav', out_wav[None], 16000)


# Torchserve Handler

In [ ]:
from ts.torch_handler.base_handler import BaseHandler
import torch

class KNN_VC_Handler(BaseHandler):
    """
    Refer to https://pytorch.org/serve/custom_service.html
    """
    def __init__(self):
        self._context = None
        self.initialized = None
        self.model = None
        self.device = None


    def initialize(self,context):
        """
        Initialize model and resources here
        """
        # From the exapmle:
        self.manifest = context.manifest
        properties = context.system_properties
        self.device = torch.device("cuda:" + str(properties.get("gpu_id")) if torch.cuda.is_available() else "cpu")

        # Download model
        self.model = torch.hub.load('bshall/knn-vc', 'knn_vc', prematched=True, trust_repo=True, pretrained=True, device=self.device)
        print(f"==================================================================================")
        print(f"==================================================================================")
        print(f"Self.model: {self.model}")
        print(f"==================================================================================")
        print(f"==================================================================================")
        self.initialized = True

    def preprocess(self, data):
        src_wav_path = data["source_path"]
        ref_wav_paths = data["target_paths"]

        query_seq = self.model.get_features(src_wav_path) # Returns features of `path` waveform as a tensor of shape (seq_len, dim) --> data preprocessing
        matching_set = self.model.get_matching_set(ref_wav_paths) # Get matching features to be used by wavlm --> Data preprocessing

        return query_seq, matching_set

    def handle(self, data:dict, context):
        """
        Invoke by TorchServe for prediction request.
        Do pre-processing of data, prediction using model and postprocessing of prediciton output
            data:   Input data for prediction
            context:    Initial context contains model server system properties.
            return: prediction output
        """

        # First pre-process the data. Accept data as a dict, and get the stuff
        query_seq, matching_set = self.preprocess(data)
        
        out_wav = self.model.match(query_seq, matching_set, topk=4)

        return out_wav




        

# Generate MAR File

#### Dummy Handler and Model:


In [ ]:
import subprocess


# command = [
#     'torch-model-archiver',
#     '--model-name', 'dummy_model',
#     '--version', '1.0',
#     '--model-file', 'dummy_model.py',
#     '--serialized-file', 'dummymodel_state',
#     '--handler', 'dummy_handler.py',
#     '--export-path', 'deployment/model_store',
#     '-f'
# ]

command = [
    'torch-model-archiver',
    '--model-name', 'knn_vc',
    '--version', '1.0',
    '--model-file', 'knn_vc/matcher.py',
    '--serialized-file', 'knn_vc_state_dict',
    '--handler', 'knn_vc_handler.py',
    '--export-path', 'deployment/model_store',
    '-f'
]
# command = ["ls"]

result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)



if result.returncode == 0:
    print("Command executed successfully.")
    print("Output:")
    print(result.stdout)
else:
    print("Command failed with the following error:")
    print(result.stderr)


<!-- torch-model-archiver --model-name knn_vc --version 1.0 --model-file knn_vc/matcher.py --serialized-file ./knn_vc_state_dict --handler knn_vc/knn_vc_handler.py -->

torch-model-archiver --model-name knn_vc --version 1.0 --model-file knn_vc/dummy_model.py --serialized-file dummymodel_state --handler knn_vc/knn_vc_handler.py --export-path deployment/model_store


# Deploy Model 

torchserve --start --ncs --model-store deployment/model_store --models knn_vc=knn_vc_test.mar

# Dummy Model

In [5]:
import torch

from knn_vc import hubconf as hc

# model = knn_vc(pretrained=True, progress=True, prematched=True, device='cuda')

# model = DummyModel()

# print(model.get_features)



ModuleNotFoundError: No module named 'wavlm'

In [ ]:
print(model.state_dict())

torch.save(model.state_dict(), "dummymodel_state")

# JSON stuff

In [ ]:
import json
import os
import torchaudio


json_file = "./datasets/debug_set/debugset.json"
with open(json_file, "r") as json_file:
    json_dict = json.loads(json_file.read()) #loads the config yaml file as a dict

print(json_dict)

print(os.path.abspath(json_dict["source_path"]))

audio = torchaudio.load(json_dict["source_path"])

print(audio)

In [ ]:
import numpy as np
import random

# def generate_json_files_for_inference():
dataset_path = "./datasets/LibriSpeech/test-clean"
num_readers = len(next(os.walk(dataset_path))[1]) # number of folders, each is a unique reader

# select a random reader
directories = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]

src_reader_id = np.random.randint(0,num_readers) #randomly select a reader id for the source
src_reader_dir = directories[src_reader_id] # reader dir for the source utterances

target_reader_id = np.random.randint(0,num_readers)

while target_reader_id == src_reader_id:
    target_reader_id = np.random.randint(0,num_readers) #so src and target aren't the same

target_reader_dir = directories[target_reader_id] # reader dir for the target utterances

# src_num_chapters_avail = len(next(os.walk(f"{dataset_path}/{src_reader_dir}"))[1]) #how many folders the src reader has to select from
# target_num_chapters_avail = len(next(os.walk(f"{dataset_path}/{target_reader_dir}"))[1]) #how many folders the target reader has to select from

src_chapter_dirs = [d for d in os.listdir(f"{dataset_path}/{src_reader_dir}") if os.path.isdir(os.path.join(f"{dataset_path}/{src_reader_dir}", d))] #the actual names of the chapter dirs for the chosen speaker
target_chapter_dirs = [d for d in os.listdir(f"{dataset_path}/{target_reader_dir}") if os.path.isdir(os.path.join(f"{dataset_path}/{target_reader_dir}", d))] #the actual names of the chapter dirs for the chosen target

src_chapter = random.choice(src_chapter_dirs)
target_chapter = random.choice(target_chapter_dirs)

src_path = f"{dataset_path}/{src_reader_dir}/{src_chapter}"
target_path = f"{dataset_path}/{target_reader_dir}/{target_chapter}"

# choosing a random .flac file for the src utterance
src_files = [f for f in os.listdir(src_path) if os.path.isfile(os.path.join(src_path, f))]
src_flac_files =[f for f in src_files if f.endswith('.flac')]
src_file = random.choice(src_flac_files)

# choosing a random number of .flac files for the target utterance
target_files = [f for f in os.listdir(target_path) if os.path.isfile(os.path.join(target_path, f))]
target_flac_files =[f for f in target_files if f.endswith('.flac')]
num_files_to_select = np.random.randint(1, len(target_flac_files)-1)
print(num_files_to_select)
print(len(target_flac_files))

target_files = random.sample(target_flac_files,num_files_to_select)

src_path_final = f"{dataset_path}/{src_reader_dir}/{src_chapter}/{src_file}"
target_paths_final = [f"{dataset_path}/{target_reader_dir}/{target_chapter}/{target_file}" for target_file in target_files]

data = {
    "source_path": src_path_final,
    "target_paths": target_paths_final,
    "working_dir": "/media/Data/CambAI/TakeHome/CambAI_TakeHome"
}

print(data)

# print(f"Choosing {src_reader_dir}/{src_chapter}/{src_file} for src")
# print(f"Choosing {num_files_to_select} files {target_reader_dir}/{target_chapter}/{target_files} for target")





# Docker


docker run --rm -it -p 127.0.0.1:8080:8080 -p 127.0.0.1:8081:8081 -p 127.0.0.1:8082:8082 --name knn_vc -v /media/Data/CambAI/TakeHome/CambAI_TakeHome/deployment/model_store:/home/model-server/model-store -v /media/Data/CambAI/TakeHome/CambAI_TakeHome/knn_vc:/home/model-server/knn_vc -v /media/Data/CambAI/TakeHome/CambAI_TakeHome/dewet_utils:/home/model-server/dewet_utils pytorch/torchserve:latest-gpu

<!-- Custom docker image -->
sudo docker run --rm -it -p 127.0.0.1:8080:8080 -p 127.0.0.1:8081:8081 -p 127.0.0.1:8082:8082 --name knn_vc -v /media/Data/CambAI/TakeHome/CambAI_TakeHome/deployment/model_store:/home/model-server/model-store -v /media/Data/CambAI/TakeHome/CambAI_TakeHome/knn_vc:/home/model-server/knn_vc -v /media/Data/CambAI/TakeHome/CambAI_TakeHome/dewet_utils:/home/model-server/dewet_utils pytorch/custom_torchserve:latest


sudo docker exec -it knn_vc /bin/bash

torch-model-archiver --model-name knn_vc --version 1.0 --model-file /home/model-server/knn_vc/matcher.py --serialized-file /home/model-server/knn_vc/state_dicts/knn_vc_state_dict --export-path /home/model-server/model-store --handler /home/model-server/dewet_utils/knn_vc_handler.py -f


<!-- Reload Hanlder -->
torch-model-archiver --model-name knn_vc --handler /home/model-server/dewet_utils/knn_vc_handler.py --export-path /home/model-server/model-store -f -v 1.0

<!-- Build custom dockerfile -->
docker build -t pytorch/custom_torchserve:latest -f Dockerfile.txt .

<!-- List dcker images -->


/home/model-server/knn_vc/state_dicts/knn_vc_state_dict.pth


curl -X POST "localhost:8081/models?model_name=knn_vc&url=/home/model-server/model-store/knn_vc.mar&initial_workers=4"